In [ ]:
!pip install openai

In [ ]:
import requests
import random
import re
import ast
from openai import OpenAI
from google.colab import userdata

#UTILS

In [ ]:
#@title board utils
def generate_board(n = 25, lang = "eng", c = 5, k = 1):
  '''
  inputs:
    n (int): number of words
    lang (str): language of dictionary (must be on the github folder)
    c (int): number of colored words for each team
    k (int): number of killer words

  returns a dictionary such that dict[word] = color
  '''

  url = f"https://raw.githubusercontent.com/mich1803/Codenames-LLM/main/wordlists/{lang}.txt"

  try:
    # Fetch the content of the file
    response = requests.get(url)
  except:
    raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")
  words = response.text.splitlines()

  # Select n unique random words
  random_words = random.sample(words, n)

  # Select indices for the colors
  indices = list(range(n))
  blue_indices = random.sample(indices, c)
  remaining_indices = [i for i in indices if i not in blue_indices]
  red_indices = random.sample(remaining_indices, c)
  remaining_indices = [i for i in remaining_indices if i not in red_indices]
  black_index = random.sample(remaining_indices, k)

  colors = [
      "blue" if i in blue_indices else
      "red" if i in red_indices else
      "killer" if i in black_index else
      "neutral"
      for i in range(n)
      ]

  # Create a dictionary with list comprehension
  word_color_dict = {random_words[i]: colors[i] for i in range(n)}
  return word_color_dict

def board4prompt(word_color_dict, lang = "eng", master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()
    lang (str): language of prompt

  return a string version for the prompt for LLMs
  '''

  if lang == "eng":
    init = "The actual word board is: \n"
  elif lang == "ita":
    init = "La board al momento è: \n"
  else: raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")

  text = "|"
  if master:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} ({word_color_dict[i]}) " + "|"
      if (idx > 0) and (((idx + 1) % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"
  else:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} " + "|"
      if (idx > 0) and ((idx % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"


  return init + text + "\n"

In [ ]:
b = generate_board(lang = "ita")
print(board4prompt(b, master = True))

The actual word board is: 
| ZIO (blue) | BOSCO (neutral) | FRUTTO (blue) | SOGNO (red) | SCENA (red) |
| POETA (neutral) | EFFETTO (neutral) | CURA (red) | POLITICA (neutral) | STRUMENTO (neutral) |
| VOLTO (blue) | PESO (neutral) | INTENZIONE (neutral) | FATTO (red) | STELLA (neutral) |
| AIUTO (neutral) | MONTE (blue) | ALBERO (neutral) | AMBIENTE (neutral) | FILM (neutral) |
| POSTO (blue) | DIRITTO (neutral) | FRATELLO (neutral) | AUTORE (killer) | ESPERIENZA (red) |




In [ ]:
#@title text-handling utils (json)
def read_clue(answer):
  try:
    data = ast.literal_eval(answer)
    return (data["clue"].upper(), data["number"])
  except:
    raise ValueError("Format error")

def read_guess(answer):
  try:
    data = ast.literal_eval(answer)
    return data["clue"].upper()
  except:
    raise ValueError("Format error")

#FIRST EXPERIMENT
First experiment to make 4 agents play (w/out history of the chat)

In [ ]:
#@title API

API_key = userdata.get('openai') #INSERIRE L'API KEY NEI SECRETS DI COLAB

def get_response_m(lang, prompt, team, model = "gpt-3.5-turbo", getprompt = False):
    if lang == "eng":
      systemprompt = "*You are a helpful assistant designed to output JSON. (DICT WITH KEYS: clue, number)* \n" + \
      "We are playing the board game CODENAMES, and your role will be the spymaster. \n" + \
      "Given a list of words, each with an associated color, you need to help your team's guessers find only the words with your team's color. \n" + \
      "YOU NEED TO PROVIDE THEM WITH A ONE-WORD CLUE (that is not on the board) and A NUMBER that represents the number of words on the board related to that clue. \n" + \
      "WARNING: if your team guesses the 'assassin' word, they lose instantly. \n" + \
      f"You are on the {team} team. \n \n"

    elif lang == "ita":
      systemprompt = "*Sei un utile assistente progettato per rispondere in JSON. (DIZIONARIO CON CHIAVI: clue, number)*" + \
      "Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del master. \n " + \
      "Data una lista con delle parole, accompagnate da un colore, devi cercare di far indovinare agli indovini della tua squadra, solo le parole con il colore della tua squadra. \n" + \
      "DEVI FORNIRNGLI UN INDIZIO DI UNA PAROLA (che non sia nella board) e UN NUMERO che rappresenta il numero di parole della board relazionate a quell'indizio. \n" + \
      "ATTENTO: se la tua squadra indovina la parola 'killer' perde istantaneamente. \n" + \
      f"Fai parte della squadra {team}. \n \n"

    else:
      raise Exception("Lingua non ancora inserita")

    if getprompt: return systemprompt

    client = OpenAI(api_key=API_key)
    chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": systemprompt},
                  {"role": "user", "content": prompt}
                  ]
    )
    return chat_completion.choices[0].message.content

def get_response_g(lang, prompt, team, model = "gpt-3.5-turbo", getprompt = False):
    if lang == "eng":
      systemprompt = "*You are a helpful assistant designed to output JSON. (DICT WITH KEY: clue)* \n" + \
      "We are playing the board game CODENAMES, and your role will be the guesser. \n" + \
      "Given a list of words and a clue from your spymaster, you need to guess ONLY ONE WORD for your team. \n"

    elif lang == "ita":
      systemprompt = "*Sei un utile assistente progettato per rispondere in JSON. (DIZIONARIO CON CHIAVE: clue)* \n" + \
      "Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del guesser. \n" + \
      "Data una lista con delle parole e un indizio da parte del tuo master devi cercare di indovinare UNA SOLA PAROLA della tua squadra. \n"
    else:
      raise Exception("Lingua non ancora inserita")

    if getprompt: return systemprompt

    client = OpenAI(api_key=API_key)
    chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": systemprompt},
                  {"role": "user", "content": prompt}
                  ]
    )
    return chat_completion.choices[0].message.content

In [ ]:
#@title prompts
prompt_colors = {
    "red": "\033[1;31m",
    "blue": "\033[1;34m",
    "endcolor": "\033[0m",
    "n": "\033[1;33m"
}

def m_prompt(lang, team, board, verbose):
  if lang == "ita":
    prompt = "È il tuo turno! " + \
            board4prompt(board, lang, master = True)

  elif lang == "eng":
    prompt = "It's your turn! " + \
            board4prompt(board, lang, master = True)


  else: raise Exception("ERROR, language not found")

  if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']} to {prompt_colors[team]}MASTER({team}){prompt_colors['endcolor']}: {prompt}")
  return prompt

def g_prompt(lang, team, board, clue, verbose):

  if lang == "ita":
    prompt = f"È il tuo turno! il tuo master ha dato l'indizio {clue}. \n" + \
            board4prompt(board, lang, master = False) + " \n" + \
            "DEVI INDOVINARE UNA SOLA PAROLA NELLA BOARD in base all'indizio fornito dal tuo master. \n \n"
  elif lang == "eng":
    prompt = f"It is your turn! your spymaster gave the clue {clue}. \n " + \
            board4prompt(board, lang, master = False) + \
            "Given your master's clue YOU NEED TO GUESS ONE WORD FROM THE BOARD. \n \n"

  else: raise Exception("!!! ERROR, language not found")

  if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']} to {prompt_colors[team]}GUESSER({team}){prompt_colors['endcolor']}: {prompt}")
  return prompt


In [ ]:
#@title play function
def play_game_GPT(lang, n, c, k, verbose=False, model_for_red = "gpt-3.5-turbo", model_for_blue = "gpt-3.5-turbo"):
    r = 1
    turn = random.choice(["red", "blue"])
    board = generate_board(n, lang, c, k)
    points = {"red": 0, "blue": 0}

    if verbose:
      print("\n----------------------\n")
      print(f"{prompt_colors['n']}GAME PARAMETERS{prompt_colors['endcolor']}: \n")
      print(f"  {n} total cards")
      print(f"  {c} coloured cards")
      print(f"  {k} killer cards")
      print(f"  {lang} language \n\n")
      print(f"{prompt_colors['red']}RED TEAM PARAMETERS{prompt_colors['endcolor']}: \n")
      print(f"  master model: {model_for_red}")
      print(f"  master system prompt: {get_response_m(lang, '', 'red', '', True)}")
      print(f"  guesser model: {model_for_red}")
      print(f"  guesser system prompt: {get_response_g(lang, '', 'red', '', True)}\n")
      print(f"{prompt_colors['blue']}BLUE TEAM PARAMETERS{prompt_colors['endcolor']}: \n")
      print(f"  master model: {model_for_red}")
      print(f"  master system prompt: {get_response_m(lang, '', 'blue', '', True)}")
      print(f"  guesser model: {model_for_red}")
      print(f"  guesser system prompt: {get_response_g(lang, '', 'blue', '', True)}")
      print("\n----------------------\n")


    while True:
        if verbose: print(f"{prompt_colors['n']} --- ROUND {r} --- {prompt_colors['endcolor']}\n \n")
        if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: it is the turn of {prompt_colors[turn]}team {turn}{prompt_colors['endcolor']}. \n \n")

        # MASTER PHASE
        model = model_for_red if turn == "red" else model_for_blue
        ans = get_response_m(lang, m_prompt(lang, turn, board, verbose), turn, model)


        try:
            clue, number = read_clue(ans)
            if verbose: print(f"{prompt_colors[turn]}MASTER({turn}){prompt_colors['endcolor']}: clue = {clue}, number = {number} \n \n")
        except:
            if verbose: print("!!! ERROR (Incorrect format of answer)")
            w = "blue" if turn == "red" else "red"
            return (w, "format eroor", r)

        if clue in board:
            w = "blue" if turn == "red" else "red"
            if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: MASTER({turn})'s clue was on the board, {prompt_colors[w]}team {w} wins{prompt_colors['endcolor']}, game ends. \n \n")
            return (w, "said a word in the board as a hint", r)

        # GUESSER PHASE
        for iter in range(number):

            model = model_for_red if turn == "red" else model_for_blue
            ans = get_response_g(lang, g_prompt(lang, turn, board, clue, verbose), turn, model)
            try:
                guess = read_guess(ans)
                if verbose: print(f"{prompt_colors[turn]}GUESSER({turn}){prompt_colors['endcolor']}: guess = {guess} \n \n")
            except:
                if verbose: print("ERROR (Incorrect format of answer) \n \n")
                w = "blue" if turn == "red" else "red"
                return (w, "format error", r)

            # EVALUATE GUESS
            try:
                x = board[guess]
            except:
                w = "blue" if turn == "red" else "red"
                if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: GUESSER({turn})'s guess wasn't on the board. \n \n")
                break

            if x == "killer":
                if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: The killer word have been selected, the game ends. \n \n ")
                w = "blue" if turn == "red" else "red"
                return (w, "killer", r)

            elif x == "blue":
                points["blue"] += 1
                if verbose: print(f'{prompt_colors["n"]}NARRATOR{prompt_colors["endcolor"]}: A {prompt_colors["blue"]}blue word{prompt_colors["endcolor"]} have been selected (blue score = {points["blue"]}). \n \n')
                del board[guess]
                if points["blue"] == c:
                    if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: The {prompt_colors['blue']}blue team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
                    return ("blue", "got all cards", r)

            elif x == "red":
                points["red"] += 1
                if verbose: print(f'{prompt_colors["n"]}NARRATOR{prompt_colors["endcolor"]}: A {prompt_colors["red"]}red word{prompt_colors["endcolor"]} have been selected (red score = {points["red"]}). \n \n')
                del board[guess]
                if points["red"] == c:
                    if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: The {prompt_colors['red']}red team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
                    return ("red", "got al cards", r)

            else:
                if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: A neutral word have been selected. \n \n")
                del board[guess]

            if x != turn: break

        turn = "blue" if turn == "red" else "red"
        r += 1

In [ ]:
#@title testing
play_game_GPT("ita", 25, 5, 1, verbose=True, model_for_red = "gpt-4o", model_for_blue = "gpt-4o")


----------------------

GAME PARAMETERS: 

  25 total cards
  5 coloured cards
  1 killer cards
  ita language 


RED TEAM PARAMETERS: 

  master model: gpt-4o
  master system prompt: *Sei un utile assistente progettato per rispondere in JSON. (DIZIONARIO CON CHIAVI: clue, number)*Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del master. 
 Data una lista con delle parole, accompagnate da un colore, devi cercare di far indovinare agli indovini della tua squadra, solo le parole con il colore della tua squadra. 
DEVI FORNIRNGLI UN INDIZIO DI UNA PAROLA (che non sia nella board) e UN NUMERO che rappresenta il numero di parole della board relazionate a quell'indizio. 
ATTENTO: se la tua squadra indovina la parola 'killer' perde istantaneamente. 
Fai parte della squadra red. 
 

  guesser model: gpt-4o
  guesser system prompt: *Sei un utile assistente progettato per rispondere in JSON. (DIZIONARIO CON CHIAVE: clue)* 
Stiamo giocando al gioco da tavola NOMI IN C

('red', 'win', 9)